URL index: [Home](https://zzz.bwh.harvard.edu/luna-walkthrough/) | [Data](https://zzz.bwh.harvard.edu/luna-walkthrough/data/) | [S1. File QC](https://zzz.bwh.harvard.edu/luna-walkthrough/p1/) | [S2. Signal QC](https://zzz.bwh.harvard.edu/luna-walkthrough/p2) | [S3. Staging](https://zzz.bwh.harvard.edu/luna-walkthrough/p3) | [S4. Artifacts](https://zzz.bwh.harvard.edu/luna-walkthrough/p4) | [S5. Analysis](https://zzz.bwh.harvard.edu/luna-walkthrough/p5)

Notebook index: [Index](../00_index.ipynb) | [S1. File QC](../p1/00_index.ipynb) | [S2. Signal QC](../p2/00_index.ipynb) | [S3. Staging](../p3/00_index.ipynb) | [S4. Artifacts](../p4/00_index.ipynb) | [S5. Analysis](../p5/00_index.ipynb)

---

# 1.4. Manipulating EDF record size

Walkthrough URL = [https://zzz.bwh.harvard.edu/luna-walkthrough/p1/recsize/](https://zzz.bwh.harvard.edu/luna-walkthrough/p1/recsize/)

In [1]:
import lunapi as lp
proj = lp.proj()
proj.sample_list( 's1.lst' )

initiated lunapi v0.1.1 <lunapi.lunapi0.luna object at 0x16a8723b0> 

read 20 individuals from s1.lst


From the prior step, we see `M06` has a nonstandard EDF record size.  We'll attach and change.

In [2]:
p = proj.inst( 'M06' )

___________________________________________________________________
Processing: M06 | ../work/data/edfs//M06.edf
 duration 07.36.44, 27404s | time 22.00.00 - 05.36.44 | date 01.01.85

 signals: 59 (of 59) selected in a standard EDF file
  FP1 | FP2 | AF3 | AF4 | F7 | F5 | F3 | F1
  F2 | F4 | F6 | F8 | FT7 | FC5 | FC3 | FC1
  FC2 | FC4 | FC6 | FT8 | T7 | C5 | C3 | C1
  C2 | C4 | C6 | T8 | TP7 | CP5 | CP3 | CP1
  CP2 | CP4 | CP6 | TP8 | P7 | P5 | P3 | P1
  P2 | P4 | P6 | P8 | PO3 | PO4 | O1 | O2
  AFZ | FZ | FCZ | CZ | CPZ | PZ | POZ | OZ
  A1 | A2 | FPZ


Chaning record size using `RECORD-SIZE`.  Note: this forces the `problem` flag to be set, so it will _look_ like an error.   Read the console log carefully and you'll see this is as expected. (`now WRITE'ing EDF to disk, and will set 'problem' flag to skip to next EDF`)

In [3]:
p.eval( 'RECORD-SIZE dur=1 edf-dir=../work/data/edfs/ edf=M06-v2' )

 ..................................................................
 CMD #1: RECORD-SIZE
   options: dur=1 edf=M06-v2 edf-dir=../work/data/edfs/ sig=*
 altering record size from 17 to 1 seconds
 now WRITE'ing EDF to disk, and will set 'problem' flag to skip to next EDF
  no epoch mask set, no restructuring needed
  data are not truly discontinuous
  writing as a standard EDF
  writing 59 channels
  saved new EDF, ../work/data/edfs/M06-v2.edf
**warning: the PROBLEM flag was set, skipping to next EDF...


RuntimeError: problem flag set: likely no unmasked records left?

As noted: `now WRITE'ing EDF to disk, and will set 'problem' flag to skip to next EDF`

That is, this is _not_ an error...

We now confirm we can load this new EDF and check the record size via `HEADERS`.  We'll create an empty (dummy) instance and attach the EDF directly (rather than working with a sample list).

In [4]:
p = proj.inst( 'dummy' )

In [5]:
p.attach_edf( '../work/data/edfs/M06-v2.edf' )

___________________________________________________________________
Processing: dummy | ../work/data/edfs/M06-v2.edf
 duration 07.36.44, 27404s | time 22.00.00 - 05.36.44 | date 01.01.85

 signals: 59 (of 59) selected in a standard EDF file
  FP1 | FP2 | AF3 | AF4 | F7 | F5 | F3 | F1
  F2 | F4 | F6 | F8 | FT7 | FC5 | FC3 | FC1
  FC2 | FC4 | FC6 | FT8 | T7 | C5 | C3 | C1
  C2 | C4 | C6 | T8 | TP7 | CP5 | CP3 | CP1
  CP2 | CP4 | CP6 | TP8 | P7 | P5 | P3 | P1
  P2 | P4 | P6 | P8 | PO3 | PO4 | O1 | O2
  AFZ | FZ | FCZ | CZ | CPZ | PZ | POZ | OZ
  A1 | A2 | FPZ


True

In [6]:
p.eval( 'HEADERS' ) 

 ..................................................................
 CMD #1: HEADERS
   options: sig=*


,Command,Strata
0,HEADERS,BL
1,HEADERS,CH


In [7]:
p.table( 'HEADERS' )

,ID,EDF_ID,EDF_TYPE,NR,NS,NS_ALL,REC_DUR,REC_DUR_HMS,REC_DUR_SEC,START_DATE,START_TIME,STOP_TIME,TOT_DUR_HMS,TOT_DUR_SEC
0,dummy,M06,EDF,27404,59,59,1.0,07:36:44,27404.0,01.01.85,22.00.00,05.36.44,07:36:44,27404.0


As expected, `REC_DUR` is now 1.0 (second).

Finally, we'll simply copy the new EDF over the existing one (so the prior `s1.lst` sample list will now point to this new EDF).

In [8]:
%%sh
mv ../work/data/edfs/M06-v2.edf ../work/data/edfs/M06.edf